# Network Intrusion Detection System

This notebook demonstrates the original intrusion detection workflow along with an enhanced model. In addition to the existing XGBoost model, we add a stacking ensemble that combines XGBoost, Random Forest, and an MLP with Logistic Regression as the final estimator to improve overall accuracy and robustness :contentReference[oaicite:0]{index=0}.

In [2]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import warnings

warnings.filterwarnings('ignore')
pd.set_option('display.float_format', lambda x: '%.3f' % x)
plt.rcParams['figure.figsize'] = (10,6)

# Load dataset (ensure 'KDDTest+.txt' is in the same directory)
df = pd.read_csv('KDDTest+.txt')

# Define column names
columns = [
    'duration', 'protocol_type', 'service', 'flag', 'src_bytes',
    'dst_bytes', 'land', 'wrong_fragment', 'urgent', 'hot', 'num_failed_logins',
    'logged_in', 'num_compromised', 'root_shell', 'su_attempted', 'num_root',
    'num_file_creations', 'num_shells', 'num_access_files', 'num_outbound_cmds',
    'is_host_login', 'is_guest_login', 'count', 'srv_count', 'serror_rate',
    'srv_serror_rate', 'rerror_rate', 'srv_rerror_rate', 'same_srv_rate',
    'diff_srv_rate', 'srv_diff_host_rate', 'dst_host_count', 'dst_host_srv_count',
    'dst_host_same_srv_rate', 'dst_host_diff_srv_rate', 'dst_host_same_src_port_rate',
    'dst_host_srv_diff_host_rate', 'dst_host_serror_rate', 'dst_host_srv_serror_rate',
    'dst_host_rerror_rate', 'dst_host_srv_rerror_rate', 'attack', 'level'
]

df.columns = columns

# Convert attack labels to binary: 'normal' vs 'attack'
df['attack'] = df['attack'].apply(lambda x: 'normal' if x == 'normal' else 'attack')

# Encode categorical features
from sklearn import preprocessing
le = preprocessing.LabelEncoder()
for col in ['protocol_type', 'service', 'flag', 'attack']:
    df[col] = le.fit_transform(df[col])

# Define features and target
X = df.drop(['attack'], axis=1)
y = df['attack']

from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.1, random_state=43)

# Select top 15 features based on previous analysis
top_features = [
    'duration', 'protocol_type', 'service', 'flag', 'src_bytes',
    'dst_bytes', 'wrong_fragment', 'hot', 'logged_in', 'num_compromised',
    'count', 'srv_count', 'serror_rate', 'srv_serror_rate', 'rerror_rate'
]

X_train = X_train[top_features]
X_test = X_test[top_features]

# Standardize features
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)


In [3]:
# Original Model: XGBoost classifier
from xgboost import XGBClassifier
from sklearn.metrics import confusion_matrix, classification_report, f1_score, recall_score, roc_auc_score

xgb_model = XGBClassifier(
    colsample_bytree=0.5,
    learning_rate=0.1,
    max_depth=6,
    n_estimators=128,
    subsample=0.8,
    random_state=42
)

xgb_model.fit(X_train, y_train)

def eval_metric(model, X_train, y_train, X_test, y_test):
    y_train_pred = model.predict(X_train)
    y_pred = model.predict(X_test)
    print('Test Set Metrics')
    print(confusion_matrix(y_test, y_pred))
    print(classification_report(y_test, y_pred))
    print('\nTrain Set Metrics')
    print(confusion_matrix(y_train, y_train_pred))
    print(classification_report(y_train, y_train_pred))

eval_metric(xgb_model, X_train, y_train, X_test, y_test)

# Additional metrics for XGBoost
y_pred = xgb_model.predict(X_test)
if hasattr(xgb_model, 'predict_proba'):
    y_pred_proba = xgb_model.predict_proba(X_test)
    f1 = f1_score(y_test, y_pred)
    recall = recall_score(y_test, y_pred)
    auc = roc_auc_score(y_test, y_pred_proba[:, 1])
    print('XGBoost F1 Score: {:.3f}'.format(f1))
    print('XGBoost Recall: {:.3f}'.format(recall))
    print('XGBoost ROC AUC: {:.3f}'.format(auc))


Test Set Metrics
[[1228   24]
 [  17  986]]
              precision    recall  f1-score   support

           0       0.99      0.98      0.98      1252
           1       0.98      0.98      0.98      1003

    accuracy                           0.98      2255
   macro avg       0.98      0.98      0.98      2255
weighted avg       0.98      0.98      0.98      2255


Train Set Metrics
[[11479   101]
 [   85  8623]]
              precision    recall  f1-score   support

           0       0.99      0.99      0.99     11580
           1       0.99      0.99      0.99      8708

    accuracy                           0.99     20288
   macro avg       0.99      0.99      0.99     20288
weighted avg       0.99      0.99      0.99     20288

XGBoost F1 Score: 0.980
XGBoost Recall: 0.983
XGBoost ROC AUC: 0.999


## Enhanced Model: Stacked Ensemble Classifier

The following cell implements a stacking ensemble that combines three base models—XGBoost, Random Forest, and an MLP classifier—with Logistic Regression as the final estimator. This approach aims to improve the detection accuracy and robustness of the intrusion detection system :contentReference[oaicite:1]{index=1}.

In [4]:
# Build and evaluate the stacking ensemble model
from sklearn.ensemble import StackingClassifier, RandomForestClassifier
from sklearn.neural_network import MLPClassifier
from xgboost import XGBClassifier
from sklearn.linear_model import LogisticRegression

estimators = [
    ('xgb', XGBClassifier(
            colsample_bytree=0.5,
            learning_rate=0.1,
            max_depth=6,
            n_estimators=128,
            subsample=0.8,
            random_state=42
         )),
    ('rf', RandomForestClassifier(
            n_estimators=100,
            max_depth=10,
            random_state=42
         )),
    ('mlp', MLPClassifier(
            hidden_layer_sizes=(50, 50),
            max_iter=300,
            random_state=42
         ))
]

final_estimator = LogisticRegression(random_state=42)

stacking_model = StackingClassifier(
    estimators=estimators,
    final_estimator=final_estimator,
    cv=5,
    n_jobs=-1
)

stacking_model.fit(X_train, y_train)

eval_metric(stacking_model, X_train, y_train, X_test, y_test)

# Additional metrics for the stacking model
y_pred = stacking_model.predict(X_test)
if hasattr(stacking_model, 'predict_proba'):
    y_pred_proba = stacking_model.predict_proba(X_test)
    f1 = f1_score(y_test, y_pred)
    recall = recall_score(y_test, y_pred)
    auc = roc_auc_score(y_test, y_pred_proba[:, 1])
    print('Stacking Model F1 Score: {:.3f}'.format(f1))
    print('Stacking Model Recall: {:.3f}'.format(recall))
    print('Stacking Model ROC AUC: {:.3f}'.format(auc))


Test Set Metrics
[[1228   24]
 [  17  986]]
              precision    recall  f1-score   support

           0       0.99      0.98      0.98      1252
           1       0.98      0.98      0.98      1003

    accuracy                           0.98      2255
   macro avg       0.98      0.98      0.98      2255
weighted avg       0.98      0.98      0.98      2255


Train Set Metrics
[[11492    88]
 [  107  8601]]
              precision    recall  f1-score   support

           0       0.99      0.99      0.99     11580
           1       0.99      0.99      0.99      8708

    accuracy                           0.99     20288
   macro avg       0.99      0.99      0.99     20288
weighted avg       0.99      0.99      0.99     20288

Stacking Model F1 Score: 0.980
Stacking Model Recall: 0.983
Stacking Model ROC AUC: 0.999


In [5]:
# Save the enhanced model to a pickle file
import pickle
import os

# Create the NSL-KDD directory if it doesn't exist
os.makedirs('ids_core/NSL-KDD', exist_ok=True)

# Path to save the model
model_path = 'ids_core/NSL-KDD/model_e.pkl'

# Save the stacking model which is our enhanced model
with open(model_path, 'wb') as f:
    pickle.dump(stacking_model, f)

print(f"Enhanced model saved to {model_path}")

Enhanced model saved to ids_core/NSL-KDD/model_e.pkl
